# Dask, Modin and Rapids

### 1) Efficiency

A small experiment using 6 datasets from Jan-Jun 2023 Yellow NYC Taxi Driver.

First, we're going to test the efficiency of Modin and Dask compared to pandas.

### 2) Dask+Modin

Modin can use Dask or Ray. In experiment 2, we're going to use Dask.

As a draft for experiment 2, we're using it for the first 6 datasets. Then we're apply it to the rest.

### 3) Dask+Modin+Rapids

First, create a draft, then apply it to all datasets.

------------------------------------------------------
### 1) Efficiency

In [1]:
import pandas as pd
import os

In [2]:
%%time
data_path = "./data"

dfs_pandas = []
for filename in os.listdir(data_path):
    filepath = os.path.join(data_path, filename)
    pandas_df = pd.read_parquet(filepath)
    dfs_pandas.append(pandas_df)

pandas_df = pd.concat(dfs_pandas, ignore_index=True)

CPU times: total: 10.7 s
Wall time: 5.13 s


In [3]:
import modin.experimental.pandas as md 

In [4]:
%%time

dfs_modin = []
for filename in os.listdir(data_path):
    filepath = os.path.join(data_path, filename)
    modin_df = md.read_parquet(filepath)
    dfs_modin.append(modin_df)

modin_df = md.concat(dfs_modin, ignore_index=True)

CPU times: total: 11.5 s
Wall time: 1min 4s


In [5]:
import dask
import dask.dataframe as dd

In [6]:
%%time

dfs_dask = []
for filename in os.listdir(data_path):
    filepath = os.path.join(data_path, filename)
    dask_df = dd.read_parquet(filepath)
    dfs_dask.append(dask_df)

dask_df = dd.concat(dfs_dask, ignore_index=True)

CPU times: total: 78.1 ms
Wall time: 450 ms


In [7]:
%%time
pandas_df.head()

CPU times: total: 0 ns
Wall time: 0 ns


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,NaN
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,NaN
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,NaN
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,NaN
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,NaN


In [8]:
%%time
modin_df.head()

CPU times: total: 15.6 ms
Wall time: 41.7 ms


2024-04-22 11:59:07,081 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:61869 (pid=26940) exceeded 95% memory budget. Restarting...
2024-04-22 11:59:07,126 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:61869' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'lambda-c461e1eb57c3bf7136f74a04f7052105', 'lambda-8da00f8fbc8df9b0685e15084b864edb', 'lambda-ce349aea9749257dc4b8c5feccf4e135', 'lambda-b8a91649da18c121f8cd57f725872fa9', 'lambda-6a6e4bc1e3e26a2ea76211731d92bd68', 'lambda-8239a06fce865f4d015279fb8fd5d911', 'lambda-186ecff0887af6df019f785a760b36b1', 'lambda-eab2e7403856c04103a89de253d4bfc1', 'lambda-24549a065363bc0ea2150704f690d783', 'lambda-aa1d9f60d90660d5895746d0ba325ff5', 'lambda-93ae5667c14d81d6af977d004c17fdb4', 'lambda-cc6224396b3c82f05799f953eb152111', 'lambda-f5978786b6d7d3040bca0ad5ccc9281d', 'lambda-6240543b002d7c8967814927802c35c9', 'lambda-d96fd8fb93e7eca1647f36476e6950b7', 'lambda-6cbd0423ce3

CancelledError: lambda-9cc13225940c7acb4e6fd271252245c4

CancelledError: lambda-f641bf72102e98599b4f97f8eda8653f

In [9]:
%%time
dask_df.head()

KilledWorker: Attempted to run task ('readparquetfsspec-a2ad8149a8c4d8bc3623322bb3ec14ce', 0) on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:61844. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

In [10]:
%%time
pd_temp = pandas_df.groupby(['StateCode']).count()

KeyError: 'StateCode'

In [11]:
%%time
md_temp = modin_df.groupby(['StateCode']).count()

KeyError: 'StateCode'

In [12]:
%%time
dd_temp = dask_df.groupby(['StateCode']).count()

KeyError: 'StateCode'

In [13]:
%%time
pandas_df.fillna({'IndividualTobaccoRate': 'Unknown'})

CPU times: total: 10.8 s
Wall time: 31.9 s


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.00,1.0,14.30,2.5,0.00,NaN
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.00,1.0,16.90,2.5,0.00,NaN
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.00,1.0,34.90,2.5,0.00,NaN
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.00,1.0,20.85,0.0,1.25,NaN
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.00,1.0,19.68,2.5,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19493615,2,2023-06-30 23:30:21,2023-07-01 00:00:46,NaN,9.20,NaN,None,42,148,0,38.53,0.00,0.5,8.51,0.00,1.0,51.04,NaN,NaN,NaN
19493616,1,2023-06-30 23:34:22,2023-07-01 00:32:59,NaN,20.20,NaN,None,132,74,0,70.00,1.75,0.5,11.97,6.55,1.0,91.77,NaN,NaN,NaN
19493617,2,2023-06-30 23:45:00,2023-07-01 00:10:00,NaN,6.16,NaN,None,256,140,0,28.03,0.00,0.5,6.41,0.00,1.0,38.44,NaN,NaN,NaN
19493618,2,2023-06-30 23:13:38,2023-06-30 23:51:19,NaN,11.65,NaN,None,91,246,0,45.74,0.00,0.5,0.00,6.55,1.0,56.29,NaN,NaN,NaN


In [14]:
%%time
modin_df.fillna({'IndividualTobaccoRate': 'Unknown'})

CPU times: total: 172 ms
Wall time: 556 ms


CancelledError: lambda-a12db2eb7114e842d30a3183c25ead49

CancelledError: lambda-e1057018be9447ea0b21f07937513528

In [15]:
%%time
dask_df.fillna({'IndividualTobaccoRate': 'Unknown'})

CPU times: total: 0 ns
Wall time: 6.37 ms


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Airport_fee
npartitions=6,,,,,,,,,,,,,,,,,,,,
,int64,datetime64[us],datetime64[us],float64,float64,float64,string,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


------------------------------------------------------
### 2) Dask+Modin